Preprocessing raw counts to anndata file

Input files: <br>
patches: $*$.jpg<br>
HE images: $*$HE.jpg<br>
Annotation files: $*$annotations.txt<br>
Metadata files: ASF_Metadata_final.csv / wtgf_metadata.xlsx<br>
Probability files: $*$Probabilities.tiff<br>
Aligned counts files: $*$stdata_under_tissue_IDs.txt<br>

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
from PIL import Image
from skimage import io
from skimage.color import rgb2gray
from sklearn import preprocessing
import skimage
from skimage import filters
import math
import scanpy as sc
import scipy.ndimage as ndi
from scipy.ndimage import gaussian_filter
from skimage import data, img_as_float, exposure, util, measure, feature, morphology, segmentation, color
from skimage.morphology import reconstruction, closing, square
from skimage.filters import sobel
from skimage.segmentation import clear_border, watershed
from skimage.measure import label, regionprops
from skimage.util import invert
from skimage.util import img_as_ubyte
from skimage.measure import label, regionprops, regionprops_table
from skimage import draw
import anndata as adata
Image.MAX_IMAGE_PIXELS = None
import gzip
import pickle
from scipy.interpolate import LinearNDInterpolator

In [2]:
def get_patch_translation(x, y, image_rescaled):
    """
    x and y mimick original patch values: 31_32
    """ 
    # get size of one patch
    testy = image_rescaled.shape[1]/32
    testx = image_rescaled.shape[0]/34

    x_indices = (x-1)*testy
    y_indices = (y-1)*testx
    
    # Make a square area of each spot
    # needs to be the same code as in cropping
    xmin = float(x_indices)-image_rescaled.shape[0]/(image_rescaled.shape[0]/100)
    ymin = float(y_indices)-image_rescaled.shape[1]/(image_rescaled.shape[0]/120)
    xmax = float(x_indices)+image_rescaled.shape[0]/(image_rescaled.shape[0]/100)
    ymax = float(y_indices)+image_rescaled.shape[1]/(image_rescaled.shape[0]/120)
 
    return x_indices, y_indices, xmin, xmax, ymin, ymax

def mask_disk(maskimg):
    """
    Makes disk inside a voxel to mimick ST spots.

    Returns a masked image minus the area outside the disk.
    """
    
    # mask a "ST spot" from the image
    rr1, cc1 = draw.ellipse(110, 100, r_radius=100, c_radius=100, shape=maskimg.shape)
    mask1 = np.zeros_like(maskimg, dtype=np.bool)
    mask1[rr1, cc1] = True
    maskimg *= mask1
    
    return maskimg

def remove_background_HE(maskimg, heimg, connectivity, min_size, stripped_name):
    """
    Removes background and gets estimate of cells close to each other 
    based on disk cut offs. There cells are "dividing" cells and will
    not be split into two.
    
    Return a binary mask, labels of individual cells in the binary mask
    and a pandas df with centroids of all nuclei in one ST spot.
    """
       
    # get threshold and filter (gets cells close together)
    thresholds = filters.threshold_multiotsu(maskimg, classes=3)
    cells = maskimg > 0.02

    # gets neighboring distances between cells
    distance = ndi.distance_transform_edt(cells)

    # sets max distance
    local_maxi = feature.peak_local_max(distance, indices=False,
                                    min_distance=connectivity)
    #labels objects at the max distance
    markers = measure.label(local_maxi)

    # segmetns cells
    segmented_cells = segmentation.watershed(-distance, markers, mask=cells)

    #remove small objects
    cleaned = morphology.remove_small_objects(segmented_cells, min_size=min_size) # 5000
    
    # gets centroids of each nuclear segment
    label_img = label(cleaned)
    regions = regionprops(label_img)
    
    #makes all the plots
    fig, ax = plt.subplots(ncols=5, figsize=(20, 10))
    ax[0].imshow(cells, cmap='gray')
    ax[0].set_title('Overlapping nuclei')
    ax[0].axis('off')
    ax[1].imshow(color.label2rgb(segmented_cells, bg_label=0))
    ax[1].set_title('Segmented nuclei')
    ax[1].axis('off')
    ax[2].imshow(color.label2rgb(cleaned, bg_label=0))
    ax[2].set_title('Cleaned image')
    ax[2].axis('off')
    ax[3].imshow(maskimg)
    ax[3].set_title('Original image')
    ax[3].axis('off')
    ax[4].imshow(heimg)
    for props in regions:
        y0, x0 = props.centroid
        ax[4].plot(round(x0), round(y0), '.g', markersize=5)
    ax[4].set_title('Segment centroids')
    ax[4].axis('off')

    plt.show()
       
    # variable with all the different segments
    labels = color.label2rgb(cleaned, bg_label=0)
    
    # df with the centroids of each nucleus
    rows = []
    for props in regions:
        y0, x0 = props.centroid
        rows.append([x0, y0])

    df = pd.DataFrame(rows, columns=["x", "y"])
    
    return cleaned, labels, df

In [4]:
# get path to training directory per image 
train_path = '/patches'
# path with original HE files
he_path = '/images'

#list all dirs in the train_path
imgs = [os.path.basename(i) for i in glob.glob(os.path.join(train_path, '*'))]

# reads in metadata for this sample to put it into a final df 
metadata = pd.read_excel('/metadata.xlsx',
                        usecols=['Genotype', 'Specimen #ID','Filename', 'Sex'])


In [9]:
# list of already processed files
h5ad_path = '/raw_data' #output folder
data_ready = [str(i).split(".h5ad")[0] for i in os.listdir(h5ad_path)]

for image_name in imgs:
    
    # check if this image has already been processed
    if (image_name in data_ready):
        continue 
        
    if not image_name in metadata['Filename'].tolist():
        continue
    
    # Get names of all patches for one array
    name =  glob.glob(os.path.join(train_path, image_name, '*jpg'))
    name_list = [str(i).split("/")[-1] for i in name]

    # Read in large HE file
    large_he_jpg_name = os.path.join(he_path, image_name + '_HE.jpg')
    heimg_large = Image.open(large_he_jpg_name)
    heimg_large = np.array(heimg_large)

    # this df collect all centroids from all patches
    centroids_df = pd.DataFrame(columns = ["patch", "centroids", "_x", "_y", "x", "y", "xy", "sample", "cell_count"])
    
    # load splotch annotations files
    splotch_ann_patch = "/annotations"
    splotch_ann_file = image_name + '_annotations.txt'
    splotch_ann = pd.read_csv(os.path.join(splotch_ann_patch, splotch_ann_file), sep = "\t")
    splotch_ann['x'] = [str(round(float(i))) for i in splotch_ann['x']]
    splotch_ann['y'] = [str(round(float(i))) for i in splotch_ann['y']]
    splotch_ann['x_y'] = [str(i)+"_"+str(j) for i,j in zip(splotch_ann['x'], splotch_ann['y'])]
    splotch_ann['patch'] = [i+"_"+j for i,j in zip(splotch_ann['image'], splotch_ann['x_y'])]
    print("Spots in annotation file...", len(splotch_ann))

    counter = 0
    for file in name_list:

        # read in probabilities from ilastik (tiff, u16) and matching HE small patch (jpg)
        stripped_name = file.split(".jpg")[0]
        prob_name = os.path.join(train_path, image_name, stripped_name + '_Probabilities.tiff')
        he_jpg_name = os.path.join(train_path, image_name, stripped_name + '.jpg')
        print("Processing patch...")
        print(stripped_name)
           
        # Load probabilities mask
        maskimg = Image.open(prob_name).convert('L')
        maskimg = np.array(maskimg)
        heimg = Image.open(he_jpg_name)
        heimg = np.array(heimg)

        # Remove backgorund of mask image
        print('Removing background, segmenting and counting...')

        # Getting ST-spot like images 
        maskimg = mask_disk(maskimg)

        # in case the disk creates and empty image, skip this patch
        if (maskimg.mean() < 0.05):
            # print("Skipping array ... No cells left after disk masking")
            cell_indicator = 0
        else:
            cell_indicator = 1
            #continue
            
        # check spot annotation value to adjust thersholding in otsu
        if (len(np.intersect1d(['muscle','interna','externa' ,'mucosae','externa and interna'],splotch_ann[splotch_ann['patch'] == stripped_name]['value'].tolist())) > 0):
            otsu_size = 50
            otsu_connectivity = 15
        else: 
            otsu_size = 50
            otsu_connectivity = 7
        
        # thresholds morphology.watershed
        if cell_indicator == 1:
            mask_bgrm, labels, centrs = remove_background_HE(maskimg, heimg, otsu_connectivity, otsu_size, stripped_name) # 1, 0

            # get x_coors and y_coors from patch name
            x_coors = str(round(float(stripped_name.split("_")[2])))
            y_coors = str(round(float(stripped_name.split("_")[3])))
            x_indices, y_indices, xtransmin, xtransmax, ytransmin, ytransmax = get_patch_translation(float(x_coors), float(y_coors), heimg_large)

            # transform centroids from patches to match centroids in the whole large HE image
            centrs['x_trans'] = centrs['x'] + xtransmin 
            centrs['y_trans'] = centrs['y'] + ytransmin 

            # make a list of centroids from a single patch
            centrs_list = pd.Series([[int(round(x)),int(round(y))] for x, y in zip(centrs['x_trans'], centrs['y_trans'])])

            # collect centroids from all patches into a pandas df
            centroids_df.at[counter, 'centroids'] = centrs_list.tolist()
            centroids_df.at[counter, 'patch'] = str(x_coors)+'_'+str(y_coors)
            centroids_df.at[counter, '_x'] = '_'+str(x_coors)
            centroids_df.at[counter, '_y'] = '_'+str(y_coors)
            centroids_df.at[counter, 'x'] = x_indices
            centroids_df.at[counter, 'y'] = y_indices
            centroids_df.at[counter, 'sample'] = image_name # change when this is finalized
            centroids_df.at[counter, 'cell_count'] =  len(centrs_list)
        else:
            # get x_coors and y_coors from patch name
            x_coors = str(round(float(stripped_name.split("_")[2])))
            y_coors = str(round(float(stripped_name.split("_")[3])))
            x_indices, y_indices, xtransmin, xtransmax, ytransmin, ytransmax = get_patch_translation(float(x_coors), float(y_coors), heimg_large)
            # collect centroids from all patches into a pandas df
            centroids_df.at[counter, 'centroids'] = []
            centroids_df.at[counter, 'patch'] = str(x_coors)+'_'+str(y_coors)
            centroids_df.at[counter, '_x'] = '_'+str(x_coors)
            centroids_df.at[counter, '_y'] = '_'+str(y_coors)
            centroids_df.at[counter, 'x'] = x_indices
            centroids_df.at[counter, 'y'] = y_indices
            centroids_df.at[counter, 'sample'] = image_name # change when this is finalized
            centroids_df.at[counter, 'cell_count'] =  0

        counter = counter + 1

    # final df
    splotch_ann.drop(["x","y"], axis = 1, inplace = True)
    df_all = pd.merge(left=splotch_ann, right=centroids_df, left_on='x_y', right_on='patch')
    df_all = pd.merge(left=df_all, right=metadata, left_on='sample', right_on='Filename')
    df_all = df_all[['sample', 'x_y', 'value', 'x', 'y', 'Genotype', 'Specimen #ID', 'cell_count', 'Sex']]
    df_all = df_all.rename(columns={"Specimen #ID": "Specimen_ID"})
    df_all = df_all.rename(columns={"x_y": "patch"})
    df_all = df_all.rename(columns={"value": "annotation"})
    
    # Lists all avaialable st data files
    counts_path = '/aligned_counts' # this is as outputed with spotter /x_y and ensums
    counts = pd.read_csv(os.path.join(counts_path, df_all['sample'][0]+'_stdata_under_tissue_IDs.txt'), sep ="\t")
    counts_df = pd.DataFrame(counts)
    counts_df.columns = [str(round(float(i.split("_")[0])))+"_"+str(round(float(i.split("_")[1]))) for i in counts_df.columns]
    
    #rename gene names
    names = pd.read_csv('/Gene_names_mm.txt', sep ="\t", header=None, names = ['gene'])
    names.index = [str(i).split('_')[0] for i in names['gene']]
    #print(names)
    names['gene_names'] = [str(i).split('_')[1] for i in names['gene']]
    names = names.drop(columns = 'gene')
    
    # merges df from pre-processing steps with the correct gene names
    # subset to only hold gene names in st data
    tmp = names[names.index.isin(counts_df.index)]
    counts_df.index = list(tmp['gene_names']) + [i for i in counts_df.index if not 'ENS' in i]
    counts_df_t = counts_df.T

    # df with st data and metadata in correct order
    df_vars = pd.merge(left=df_all, right=counts_df_t, left_on='patch', right_index=True)
    df_vars.index = [str(i)+'_'+str(j) for i,j in zip(df_vars['sample'],df_vars['patch'])]
    #print("Spots after merging counts...", len(df_vars))

    # df with st gene data in correct order
    df_genes = df_vars.iloc[:,9:]
    df_genes.index = [str(i)+'_'+str(j) for i,j in zip(df_vars['sample'],df_vars['patch'])]

    # make anndata object that with go into tangram
    space_anndata = adata.AnnData(obs = df_vars.iloc[:,:10].astype({'x': 'int64', 'y': 'int64', 'cell_count': 'int64'}), X = df_genes)

    # makes sure all gene names are unique
    space_anndata.var_names_make_unique()

    # makes sure the obs_names does not start with a number as that isn't supported in anndata
    space_anndata.obs_names = [str(i)+'_'+str(j) for i,j in zip(space_anndata.obs['sample'],space_anndata.obs['patch'])]

    # collects total gene counts in a new variable used later for filtering
    space_anndata.var['n_counts'] =[int(i) for i in space_anndata.X.sum(axis=0)]

    # writes h5ad file 
    space_anndata.write_h5ad(filename = os.path.join(h5ad_path, image_name+'.h5ad'))




In [10]:
# checks if any files are missing based on metadata
if len(data_ready) == len(metadata.index):
    print("all data segmented ...")
else: 
    print("These files don't match: ")
    print(np.setdiff1d(metadata['Filename'], data_ready))


all data segmented ...
